In [ ]:
import os
import time
import pickle
import shutil
import random
import urllib
import requests
import concurrent.futures
from  urllib.request import urlopen
from termcolor import colored
from utils import Config, safe_pickle_dump

printgreen = lambda q: print(colored(q, 'green'))
printred = lambda q: print(colored(q, 'red'))

session = requests.session()
session.proxies = {
    'http': 'socks5://localhost:9050',
    'https': 'socks5://localhost:9050'
}
headers = {'User-agent': 'HotJava/1.1.2 FCS'}

In [ ]:
req = session.get('https://api.ipify.org', timeout = 15)
req.content
#with open(fname, 'wb') as fp:
#shutil.copyfileobj(req.raw, fp)

In [ ]:
db = pickle.load(open(Config.db_path, 'rb'))

In [ ]:
have = set(os.listdir(Config.pdf_dir))

In [ ]:
print(len(db))
print(len(have))

print(len(db) - len(have))

In [ ]:
have = set(os.listdir(Config.pdf_dir))

def refreshNeeded():
    entries = []
    for db_key, j in db.items():
        pdfs = [x['href'] for x in j['links'] if x['type'] == 'application/pdf']
        assert len(pdfs) == 1
        if ('ishtml' in db[db_key] and db[db_key]['ishtml'] == True):
            #print(db_key)
            continue
        pdf_url = pdfs[0] + '.pdf'
        basename = pdf_url.split('/')[-1]
        fname = os.path.join(Config.pdf_dir, basename)
        if not basename in have:
            entries.append({
                'key': db_key, 
                'url': pdf_url, 
                'basename': basename, 
                'filename': fname
            })
    return entries

newEntries = refreshNeeded()

print(len(newEntries))

In [ ]:
print(newEntries[0])
print(list(have)[0] if len(list(have)) !=0 else "0")

In [ ]:
def load_url(pdf_url):
    return session.get(pdf_url, timeout = 30)

def reqToFile(db_key, basename, filename, req):
    if (req.ok and req.headers['Content-Type'] == 'application/pdf'):
        with open(filename, 'wb') as f:
            f.write(req.content)
        print(".", end='', flush=True)
    elif ('text/html' in req.headers['Content-Type']):
        db[db_key]['ishtml'] = True
        raise ValueError(f"{db_key}: was HTML instead of PDF")
    else:
        raise Exception(f"{db_key}: Unknown error")

In [ ]:
have = set(os.listdir(Config.pdf_dir))
newEntries = refreshNeeded()

with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:
    
    future_to_url = { executor.submit(load_url, entry['url']): entry for entry in newEntries }
    
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            this_entry = future_to_url[future]
            req = future.result()
            reqToFile(this_entry['key'], this_entry['basename'], this_entry['filename'], req)
        except Exception as e:
            print('')
            print(e)
            print('')
safe_pickle_dump(db, Config.db_path)
print("Done")